In [1]:
import pickle
import pandas as pd
from pprint import pprint
from collections import defaultdict

### Read author file

In [2]:
with open("authors_gscholar_scrapped.pickle", "rb") as f:
    all_authors_gs = pickle.load(f)

print(len(all_authors_gs))

1404


In [3]:
list(all_authors_gs.keys())[0:5]

['', 'Max Jaderberg', 'Moshe Looks', 'Todd Mytkowicz', 'Stefano Soatto']

In [5]:
len(all_authors_gs["Max Jaderberg"])

1

In [11]:
print(all_authors_gs["Max Jaderberg"][0][0])

{'_filled': True,
 'affiliation': 'DeepMind',
 'citedby': 7878,
 'citedby5y': 7835,
 'cites_per_year': {2014: 24,
                    2015: 134,
                    2016: 565,
                    2017: 1085,
                    2018: 1944,
                    2019: 2958,
                    2020: 1092},
 'coauthors': [<scholarly.scholarly.Author object at 0x7f47015ddbe0>,
 'email': '@robots.ox.ac.uk',
 'hindex': 21,
 'hindex5y': 21,
 'i10index': 22,
 'i10index5y': 22,
 'id': 'JeZoMicAAAAJ',
 'interests': ['Machine Learning'],
 'name': 'Max Jaderberg',
 'publications': [<scholarly.scholarly.Publication object at 0x7f47015e50b8>,
 'url_picture': 'https://scholar.google.com/citations?view_op=medium_photo&user=JeZoMicAAAAJ'}


In [31]:
for pp in all_authors_gs["Max Jaderberg"][0][0].publications:
    print(pp, pp.bib)
    break

{'_filled': False,
 'bib': {'title': 'Spatial Transformer Networks', 'year': 2015},
 'citedby': 2849,
 'id_citations': 'JeZoMicAAAAJ:L8Ckcad2t8MC',
 'source': 'citations'} {'title': 'Spatial Transformer Networks', 'year': 2015}


### Add conflicts data

In [13]:
data_path = "../../ICLR data/masterdata_unbalanced/"

years = [2017]

for y in years:
    papers_data = pd.read_pickle(data_path + "papers_{}.pkl".format(y))

In [14]:
conflicts_dict = {}
emails_dict = {}

for p in papers_data:
    conflicts_dict[p] = papers_data[p]["content"]["conflicts"]
    emails_dict[p] = papers_data[p]["content"]["authorids"]

### Read features

In [15]:
data_raw = pd.read_pickle('../features/data_features_csv-2017.pkl')
data = data_raw[["id", "title", "label", "authors"]]
org_papers = data.to_dict('index')

### Disambiguate authors based on email

In [37]:
paper_authors_info = defaultdict(dict)

for p in org_papers:
    for auth in org_papers[p]["authors"]:
        if auth in all_authors_gs:
            if len(all_authors_gs[auth]) == 1:
                disambiguated_author = all_authors_gs[auth]
            else:
                #print(p, auth, conflicts_dict[p], emails_dict[p])
                #print("--------------------------------------------------------------------------------------")
                for mauth in all_authors_gs[auth]:
                    #print(mauth[0].name, mauth[0].email, mauth[0].interests)
                    found_paper = False
                    for pub in mauth[0].publications:
                        org_title = ''.join(filter(str.isalpha, org_papers[p]["title"].lower()))
                        gs_pub_title = ''.join(filter(str.isalpha, pub.bib["title"].lower()))
                        if org_title == gs_pub_title:
                            found_paper = True
                            disambiguated_author = mauth[0]
                            break
                    if not found_paper:

ByBwSPcex Hang Chu ['cs.toronto.edu'] ['chuhang1122@cs.toronto.edu', 'urtasun@cs.toronto.edu', 'fidler@cs.toronto.edu']
--------------------------------------------------------------------------------------
Chun-Hung Chu @hku.hk ['Faculty of Dentistry', 'The University of Hong Kong']


Vinh-Hung Vincent @chu-martinique.fr []


Hung-Kuo Chu @cs.nthu.edu.tw ['Shape Analysis', 'Smart Manipulation', 'Non-Photorealistic Rendering', 'Human Computer Interaction', 'Visual Perception.']


Hung-Chu Hsu @mail.nsysu.edu.tw ['Nonlinear water waves', 'Coastal engineering']


Nguyen Van Toan  ['thời trang']


Jian-Hung Chen @g.chu.edu.tw []


CHU Manh Hung @itims.edu.vn ['semiconductor nanomaterials', '2D thin film materials', 'nanodevice applications and synchrotron source based techniques']


Hung-Chi Chu (朱鴻棋) @cyut.edu.tw ['Wireless Network', 'Wireless Sensor Networks', 'Artificial Intelligence', 'Internet of Things']


Chien-Hung, Wu @chu.edu.tw ['半導體物理、感測器應用']


Hang Chu @cornell.edu ['Computer

In [ ]:
for p in org_papers:
    paperid = p
    for auth in org_papers[p]["authors"]:
        